In [116]:
import numpy as np
import scipy.integrate as integrate
from numpy.linalg import inv
import math as m
from functions import *
import importlib

%load_ext autoreload

%autoreload 2

np.set_printoptions(suppress=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This next block will create a class for all of our inputs to be placed into. This will allow for all inputs to have easily accessed names, values, and units.

In [117]:
class Input:
    def __init__(self, name, value, unit):
        self.name = name
        self.value = value
        #self.valueType = type(value)
        self.unit = unit
    
    def printValues(self):
        print("\n")
        for attr, value in self.__dict__.items():
            print(f"{attr}: {value}")

# Please fill in all inputs and their units below
# v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v 

For Reference: Units Set: MPa, mm2, mm, N

In [118]:
YoungsModulus = 120000
YM_Unit = "MPa"

Area = 500
A_Unit = "mm2"

P = -10000
Force_Unit = "N"

# node connections for each element
nArray = np.array([[1,2], [1,3], [1,4], [2,4], [3,4], [3,5], [3,6], [4,6], [5,6], [5,7], [5,8], [6,8], [7,8], [7,9], [8,9]])

# list of lengths corresponding to each element
# the length of element n is contained in lengths
lArray = np.array([5, 3, 4.8023431781, 3.25, 3.75, 3, 3.905124838, 3.25, 2.5, 3, 3.25, 3.25, 1.25, 3, 3.25])
lArray *= 1000

Length_unit = "mm"

# list of angles relative to positive x axis for each element
theta = m.asin(5/13)
aArray = np.array([(1.5*m.pi), 0, -(m.atan(3.75/3)), theta, 1.5*m.pi, 0, -(m.atan(2.5/3)), theta, 1.5*m.pi, 0, -(m.atan(1.25/3)), theta, 1.5*m.pi, 0, theta])

Angle_unit = "rad"

# input boundary conditions below
# for displacements: [node#, u, v]
# for forces: [node#, Fx, Fy]

uv_BCs = np.array([[1, 0, 0],[2, 0, None]])

force_BCs = np.array([[1, None, None], [2, None, 0], [3, 0, P],[5, 0, P], [7, 0, P], [9, 0, P]])

if len(aArray) != len(lArray) or len(nArray) != len(lArray) or len(nArray) != len(aArray):
    print("Error! Inputs do not have the same amount of nodes or elements!")

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [119]:
E = Input("Young's Modulus", YoungsModulus, YM_Unit)

A = Input("Area", Area, A_Unit)

nodes = Input("Node numbers", nArray, "None")
nodes.printValues()

lengths = Input("Lengths", lArray, Length_unit)
lengths.printValues()

angles = Input("Element Angles (relative to the positive x axis)", aArray, Angle_unit)
angles.printValues()



name: Node numbers
value: [[1 2]
 [1 3]
 [1 4]
 [2 4]
 [3 4]
 [3 5]
 [3 6]
 [4 6]
 [5 6]
 [5 7]
 [5 8]
 [6 8]
 [7 8]
 [7 9]
 [8 9]]
unit: None


name: Lengths
value: [5000.        3000.        4802.3431781 3250.        3750.
 3000.        3905.124838  3250.        2500.        3000.
 3250.        3250.        1250.        3000.        3250.       ]
unit: mm


name: Element Angles (relative to the positive x axis)
value: [ 4.71238898  0.         -0.89605538  0.39479112  4.71238898  0.
 -0.69473828  0.39479112  4.71238898  0.         -0.39479112  0.39479112
  4.71238898  0.          0.39479112]
unit: rad


number of elements is the number of rows in nodes
<br>number of nodes is the maximum number found in nodes

In [120]:
numElems = len(nodes.value)
print(f"Number of elements: {numElems}")
numNodes = nodes.value.max()
print(f"Number of nodes: {numNodes}")

Number of elements: 15
Number of nodes: 9


two reactions per node
<br>uvList -> list of unknowns (node displacements)
<br>we can input our known values -> fixed_nodes at nodes 4 & 5
<br>these fixed nodes numbers are taken straight from diagram (not adjusted for python counting)
<br><br>u and v for any given node is given as (node# - 1)* 2 and ((node# -1)* 2) + 1 respectively.


In [121]:
uvList = [None]*(numNodes*2)
print("displacement bcs: [node number, x disp (u), y disp (v)]")
print(uv_BCs)

adjust_array(uvList, uv_BCs, "displacements")
print("u & v list:")
print(uvList)

displacement bcs: [node number, x disp (u), y disp (v)]
[[1 0 0]
 [2 0 None]]
adjusting array to account for: displacements boundary conditions
case 1 for BC 1
Case 2 for BC 1
case 1 for BC 2
case 4 for BC 2
done adjusting displacements array!
u & v list:
[0, 0, 0, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


Same procedure as displacement BCs for forces
<br>Input BCs given as : [node #, x force, y force]

In [122]:
#forceList = [0]*(numNodes*2)
forceList = np.zeros(((numNodes*2), 1))

nodesWithLoads = []

loadedNodes = 0

for i in range(len(force_BCs)):
    if (force_BCs[i,1]!= None and force_BCs[i,1] != 0) or (force_BCs[i,2] != None and force_BCs[i,2] != 0):
        loadedNodes += 1
        nodesWithLoads.append((force_BCs[i,0] - 1))


print(f"number of nodes with concentrated loads: {loadedNodes}")
print(f"at nodes: {nodesWithLoads} (Python adjusted nodes)")

#adjust force array given concentrated loading conditions
adjust_array(forceList, force_BCs, "forces")

print(f"force list: \n{forceList}")

number of nodes with concentrated loads: 4
at nodes: [2, 4, 6, 8] (Python adjusted nodes)
adjusting array to account for: forces boundary conditions
case 1 for BC 1
Case 2 for BC 1
case 1 for BC 2
case 4 for BC 2
case 3 for BC 3
Case 2 for BC 3
case 3 for BC 4
Case 2 for BC 4
case 3 for BC 5
Case 2 for BC 5
case 3 for BC 6
Case 2 for BC 6
done adjusting forces array!
force list: 
[[    nan]
 [    nan]
 [    nan]
 [     0.]
 [     0.]
 [-10000.]
 [     0.]
 [     0.]
 [     0.]
 [-10000.]
 [     0.]
 [     0.]
 [     0.]
 [-10000.]
 [     0.]
 [     0.]
 [     0.]
 [-10000.]]


Global [k] matrix is a symmetric array of shape (numNodes x 2, numNodes x 2)
<br>Bar Constants are one per element, stored in a numElemes * 1 array.

In [123]:
globalK = np.zeros(((numNodes*2),(numNodes*2)))
print(f"Global K array dimensions: {globalK.shape}")

barConstant = np.zeros((numElems,1))
for i in range(numElems):
    barConstant[i] = (E.value*A.value)/lengths.value[i]

print(f"Bar Constants = \n{barConstant}")

Global K array dimensions: (18, 18)
Bar Constants = 
[[12000.        ]
 [20000.        ]
 [12493.90095102]
 [18461.53846154]
 [16000.        ]
 [20000.        ]
 [15364.42559176]
 [18461.53846154]
 [24000.        ]
 [20000.        ]
 [18461.53846154]
 [18461.53846154]
 [48000.        ]
 [20000.        ]
 [18461.53846154]]


## Creating the finite element equation for each Truss Element
### Combining into the global K array

In [124]:
for i in range(numElems):
    kMatrix = create_truss_k(angles.value[i])
    kMatrix *= barConstant[i]
    print(f"\nk matrix for element {i+1}: \n{kMatrix}")
    #print(kMatrix)
    startNode = nodes.value[i,0]
    endNode = nodes.value[i,1]
    print(f"Element {i+1} starting at node: {startNode} and ending at node: {endNode}")
    #print(np.vsplit(kMatrix, 2))
    Sn = (startNode - 1)*2
    En = (endNode - 1)*2
    #first, adjust 0,0 -> then, 2,0; 0,2; and 2,2
    block_array_adjust(Sn, Sn, globalK, 0, 0, kMatrix)
    block_array_adjust(En, Sn, globalK, 2, 0, kMatrix)
    block_array_adjust(Sn, En, globalK, 0, 2, kMatrix)
    block_array_adjust(En, En, globalK, 2, 2, kMatrix)

print(f"\nGlobal K matrix: \n{globalK}")
unmodified_globalK = globalK.copy()
check_Symmetric(globalK)


k matrix for element 1: 
[[     0.      0.     -0.     -0.]
 [     0.  12000.     -0. -12000.]
 [    -0.     -0.      0.      0.]
 [    -0. -12000.      0.  12000.]]
Element 1 starting at node: 1 and ending at node: 2

k matrix for element 2: 
[[ 20000.      0. -20000.     -0.]
 [     0.      0.     -0.     -0.]
 [-20000.     -0.  20000.      0.]
 [    -0.     -0.      0.      0.]]
Element 2 starting at node: 1 and ending at node: 3

k matrix for element 3: 
[[ 4875.66866381 -6094.58582977 -4875.66866381  6094.58582977]
 [-6094.58582977  7618.23228721  6094.58582977 -7618.23228721]
 [-4875.66866381  6094.58582977  4875.66866381 -6094.58582977]
 [ 6094.58582977 -7618.23228721 -6094.58582977  7618.23228721]]
Element 3 starting at node: 1 and ending at node: 4

k matrix for element 4: 
[[ 15730.5416477    6554.39235321 -15730.5416477   -6554.39235321]
 [  6554.39235321   2730.99681384  -6554.39235321  -2730.99681384]
 [-15730.5416477   -6554.39235321  15730.5416477    6554.39235321]
 [ -

In [125]:
j = 0
while j < numNodes*2:
    print(f"{j}")
    #if j in nodesWithLoads:
     #   j+=2
    #first case: displacement BC = 0. In this case, set combined force values to 0, and alter 
    #globalK to reflect the changes.
    if uvList[j] == 0:
        print(f"u = 0 at row: {j}")
        forceList[j] = 0 
        globalK[j] = 0
        globalK[j,j] = 1
        j+=1
    elif uvList[j] == None:
        print(f"none detected at row {j}")
        j+=1
    else: #when we have a known, non-zero displacement boundary condition
        #alter global stiffness to all zeros and one 1, and set force equal to displacement BC
        print(f"row {j} altered")
        print(uvList[j])
        forceList[j] = uvList[j]
        for c in range(numNodes):
            if (globalK[j,c] > 0) == True:
                globalK[j,c] = 1
            else:
                globalK[j,c] = 0
        j+=1

                
print(globalK)
print(forceList)

0
u = 0 at row: 0
1
u = 0 at row: 1
2
u = 0 at row: 2
3
none detected at row 3
4
none detected at row 4
5
none detected at row 5
6
none detected at row 6
7
none detected at row 7
8
none detected at row 8
9
none detected at row 9
10
none detected at row 10
11
none detected at row 11
12
none detected at row 12
13
none detected at row 13
14
none detected at row 14
15
none detected at row 15
16
none detected at row 16
17
none detected at row 17
[[     1.              0.              0.              0.
       0.              0.              0.              0.
       0.              0.              0.              0.
       0.              0.              0.              0.
       0.              0.        ]
 [     0.              1.              0.              0.
       0.              0.              0.              0.
       0.              0.              0.              0.
       0.              0.              0.              0.
       0.              0.        ]
 [     0.            

In [126]:
#globalK.round(0)
globalK_inv = inv(globalK)
#print(globalK_inv)

node = 1

uFinal = np.matmul(globalK_inv, forceList)

for i in range(len(uFinal)):
    if check_even(i) == True:
        print(f"u{node} = {uFinal[i]}")
    else:
        print(f"v{node} = {uFinal[i]}")
        node += 1 

u1 = [-0.]
v1 = [-0.]
u2 = [-0.]
v2 = [-2.08333333]
u3 = [2.4]
v3 = [-5.53609553]
u4 = [-2.89641853]
v4 = [-4.28609553]
u5 = [4.2]
v5 = [-13.48139753]
u6 = [-2.37684825]
v6 = [-12.85639753]
u7 = [5.4]
v7 = [-24.93111543]
u8 = [0.27893699]
v8 = [-24.7227821]
u9 = [6.6]
v9 = [-43.555]


Forces can be found from [K]*[u]

In [127]:
forcesFinal = np.matmul(unmodified_globalK, uFinal)
for i in range(len(forceList)):
    if forceList[i] != forcesFinal[i] and forceList[i] != 0 and forceList[i] != None:
        forcesFinal[i] = forceList[i]
print(f"Final Forces: \n{forcesFinal}")

node = 1

for i in range(len(forcesFinal)):
    if check_even(i) == True:
        print(f"fx{node} = {forcesFinal[i]}")
    else:
        print(f"fy{node} = {forcesFinal[i]}")
        node += 1 


Final Forces: 
[[-60000.]
 [ 40000.]
 [ 60000.]
 [    -0.]
 [    -0.]
 [-10000.]
 [     0.]
 [     0.]
 [     0.]
 [-10000.]
 [     0.]
 [     0.]
 [     0.]
 [-10000.]
 [     0.]
 [    -0.]
 [     0.]
 [-10000.]]
fx1 = [-60000.]
fy1 = [40000.]
fx2 = [60000.]
fy2 = [-0.]
fx3 = [-0.]
fy3 = [-10000.]
fx4 = [0.]
fy4 = [0.]
fx5 = [0.]
fy5 = [-10000.]
fx6 = [0.]
fy6 = [0.]
fx7 = [0.]
fy7 = [-10000.]
fx8 = [0.]
fy8 = [-0.]
fx9 = [0.]
fy9 = [-10000.]


## Finding Stresses

In [128]:
stressFinal = np.zeros((numElems, 1))
print(stressFinal)
for i in range(numElems):
    startNode = nodes.value[i,0]
    endNode = nodes.value[i,1]
    print(f"Element {i+1} starts at node: {startNode} and ends at node: {endNode}")
    #print(np.vsplit(kMatrix, 2))
    Sn = (startNode - 1)*2
    En = (endNode - 1)*2
    print(f"adjusted start node: {Sn} adjusted end node: {En}")
    trigMatrix = create_truss_stress_trig_matrix(angles.value[i])
    print(f"trigMatrix = \n{trigMatrix}")
    local_uMatrix = np.array([
        uFinal[Sn], 
        uFinal[(Sn + 1)], 
        uFinal[En], 
        uFinal[(En+1)]
        ])
    print(f"local u matrix: \n{local_uMatrix}")
    localLength = np.array([[(-1/lengths.value[i]), 1/lengths.value[i]]])
    print(f"local length matrix: \n{localLength}")

    indStress = (trigMatrix @ local_uMatrix)
    indStress = localLength @ indStress
    indStress *= E.value

    stressFinal[i] = indStress
    
    
    print(f"Stress in element = {indStress}")

    

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Element 1 starts at node: 1 and ends at node: 2
adjusted start node: 0 adjusted end node: 2
trigMatrix = 
[[-0. -1.  0.  0.]
 [ 0.  0. -0. -1.]]
local u matrix: 
[[-0.        ]
 [-0.        ]
 [-0.        ]
 [-2.08333333]]
local length matrix: 
[[-0.0002  0.0002]]
Stress in element = [[50.]]
Element 2 starts at node: 1 and ends at node: 3
adjusted start node: 0 adjusted end node: 4
trigMatrix = 
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]]
local u matrix: 
[[-0.        ]
 [-0.        ]
 [ 2.4       ]
 [-5.53609553]]
local length matrix: 
[[-0.00033333  0.00033333]]
Stress in element = [[96.]]
Element 3 starts at node: 1 and ends at node: 4
adjusted start node: 0 adjusted end node: 6
trigMatrix = 
[[ 0.62469505 -0.78086881  0.          0.        ]
 [ 0.          0.          0.62469505 -0.78086881]]
local u matrix: 
[[-0.        ]
 [-0.        ]
 [-2.89641853]
 [-4.28609553]]
local length matrix: 
[[-0.00020823 

In [129]:
print(stressFinal)

[[  50.        ]
 [  96.        ]
 [  38.41874542]
 [-130.        ]
 [ -40.        ]
 [  72.        ]
 [  31.2409987 ]
 [-104.        ]
 [ -30.        ]
 [  48.        ]
 [  26.        ]
 [ -78.        ]
 [ -20.        ]
 [  48.        ]
 [ -52.        ]]


In [130]:
np.set_printoptions(suppress=True, formatter={'float':"{:0.4f}".format})
printTrussNodalDisplacements(uFinal, lengths.unit)
printStresses(stressFinal,E.unit)

████████████████████████████████████████████████████████████████████████████████████████████████████
The nodal displacements (u,v), in [mm], ordered from lowest to highest numbered node are:
[[-0.0000]
 [-0.0000]
 [-0.0000]
 [-2.0833]
 [2.4000]
 [-5.5361]
 [-2.8964]
 [-4.2861]
 [4.2000]
 [-13.4814]
 [-2.3768]
 [-12.8564]
 [5.4000]
 [-24.9311]
 [0.2789]
 [-24.7228]
 [6.6000]
 [-43.5550]]
████████████████████████████████████████████████████████████████████████████████████████████████████
████████████████████████████████████████████████████████████████████████████████████████████████████

The element axial stresses, in [MPa], ordered from lowest to highest numbered element are:
[[50.0000]
 [96.0000]
 [38.4187]
 [-130.0000]
 [-40.0000]
 [72.0000]
 [31.2410]
 [-104.0000]
 [-30.0000]
 [48.0000]
 [26.0000]
 [-78.0000]
 [-20.0000]
 [48.0000]
 [-52.0000]]

Note: a (+) value indicates the element is in tension, while a (-) value in compression.

██████████████████████████████████████████████████